In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os
import math
import random
import scipy as sp
import scipy.stats

from torch.utils.data import DataLoader,Dataset
import h5py
import matplotlib.pyplot as plt
from torch.utils.data.sampler import Sampler

from pyts.image import RecurrencePlot
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d
from sklearn.preprocessing import OneHotEncoder
import warnings

In [2]:
# task_generator
def mini_imagenet_folders(): # train valid test split
    # temp_meatrain = ['8PSK','AM-DSB-WC','BPSK','GMSK','16APSK','64QAM']
    # temp_metavalid=['QPSK','AM-SSB-WC','16QAM','32PSK','FM']
    # temp_metatest=['OOK','4ASK','16PSK','32APSK','32QAM']
    # metatrain_folders=le.transform(temp_meatrain)
    # metavalid_folders = le.transform(temp_metavalid)
    # metatest_folders=le.transform(temp_metatest)
    metatrain_folders = np.random.choice(classes, size=TRAIN_NUM, replace=False)
    metavalid_folders = np.random.choice(list(set(classes)-set(metatrain_folders)), size=VALID_NUM, replace=False)
    metatest_folders = list(set(classes)-set(metatrain_folders)-set(metavalid_folders))
    # metatest_folders = list(set(classes)-set(metatrain_folders))
    return list(metatrain_folders), list(metavalid_folders), metatest_folders
    # return list(metatrain_folders), list(metatest_folders)

class MiniImagenetTask(object):
    # support set, query image sampling
    def __init__(self, character_folders, num_classes, train_num, test_num):
        self.character_folders = character_folders # mods
        self.num_classes = num_classes # way
        self.train_num = train_num # shot
        self.test_num = test_num # query

        class_folders = random.sample(self.character_folders,self.num_classes)
        # print(class_folders)
        self.train_roots= np.array([]).reshape(0,2,len_samples,len_samples)
        self.test_roots= np.array([]).reshape(0,2,len_samples,len_samples)
        self.train_labels = []
        self.test_labels = []
        label_enc = MyLabelEncoder()
        for c in class_folders:
            sample_idx = np.where(Y==c)
            # print(sample_idx)
            samples = X[sample_idx]
            # print(samples.shape)
            labels = Y[sample_idx]
            
            label_enc.fit(labels)
            labels = label_enc.transform(labels)
            train_samples, test_samples, train_samples_labels, test_samples_labels = train_test_split(
                samples, labels, test_size=test_num, train_size=train_num,stratify=labels)
    
            self.train_roots = np.vstack((self.train_roots,train_samples))
            self.test_roots = np.vstack((self.test_roots,test_samples))
            self.train_labels.extend(train_samples_labels)
            self.test_labels.extend(test_samples_labels)
            
        
        # self.train_roots = np.array(self.train_roots)
        # self.test_roots = np.array(self.test_roots)
        # self.train_labels = np.array(self.train_labels)
        # self.test_labels = np.array(self.test_labels)

class FewShotDataset(Dataset):
    def __init__(self, task, split='train', transform=None, target_transform=None):
        # self.transform = transform # Torch operations on the input image
        # self.target_transform = target_transform
        self.task = task
        self.split = split
        self.image_roots = self.task.train_roots if self.split == 'train' else self.task.test_roots
        self.labels = self.task.train_labels if self.split == 'train' else self.task.test_labels
    def __len__(self):
        return len(self.image_roots)
    def __getitem__(self, idx):
        raise NotImplementedError("This is an abstract class. Subclass this class for your particular dataset.")

class MiniImagenet(FewShotDataset):
    def __init__(self, *args, **kwargs):
        super(MiniImagenet, self).__init__(*args, **kwargs)
    def __getitem__(self, idx):
        image = torch.tensor(self.image_roots[idx], dtype=torch.float)
        # if self.transform is not None:
        #     image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.int)
        # if self.target_transform is not None:
        #     label = self.target_transform(label)
        return image, label

class ClassBalancedSampler(Sampler):
    ''' Samples 'num_inst' examples each from 'num_cl' pools
        of examples of size 'num_per_class' '''
    def __init__(self, num_per_class, num_cl, num_inst,shuffle=True):
        self.num_per_class = num_per_class
        self.num_cl = num_cl
        self.num_inst = num_inst
        self.shuffle = shuffle
    
    def __iter__(self):
        # return a single list of indices, assuming that items will be grouped by class\n",
        if self.shuffle:
            batch = [[i+j*self.num_inst for i in torch.randperm(self.num_inst)[:self.num_per_class]] for j in range(self.num_cl)]
        else:
            batch = [[i+j*self.num_inst for i in range(self.num_inst)[:self.num_per_class]] for j in range(self.num_cl)]
        batch = [item for sublist in batch for item in sublist]
        if self.shuffle:
            random.shuffle(batch)
        return iter(batch)
    
    def __len__(self):
        return 1

def get_mini_imagenet_data_loader(task, num_per_class=1, split='train',shuffle = False):
    # dataset = MiniImagenet(task,split=split,transform=transforms.Compose([transforms.ToTensor()]))
    dataset = MiniImagenet(task,split=split)
    if split == 'train':
        sampler = ClassBalancedSampler(num_per_class,task.num_classes, task.train_num,shuffle=shuffle)
    
    else:
        sampler = ClassBalancedSampler(num_per_class,task.num_classes, task.test_num,shuffle=shuffle)
    # num_workers = (CPU core / GPU core) - 2
    loader = DataLoader(dataset, batch_size=num_per_class*task.num_classes, sampler=sampler, pin_memory = True, num_workers=10)
    return loader

In [3]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m,h
    
class CNNEncoder(nn.Module):
    """docstring for ClassName"""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(in_channels=2,out_channels=64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(num_features=64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())
        self.layer4 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())

    def forward(self,x):
        out = self.layer1(x)
        # print('CNN Layer 1 out is {}'.format(out.shape))
        out = self.layer2(out)
        # print('CNN Layer 2 out is {}'.format(out.shape))
        out = self.layer3(out)
        # print('CNN Layer 3 out is {}'.format(out.shape))
        out = self.layer4(out)
        # print('CNN Layer 4 out is {}'.format(out.shape))
        #out = out.view(out.size(0),-1)
        # print(out.shape)
        return out # 64

class RelationNetwork(nn.Module):
    """docstring for RelationNetwork"""
    def __init__(self,input_size,hidden_size):
        super(RelationNetwork, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(64*2,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.fc1 = nn.Linear(input_size*3*3,hidden_size)
        self.fc2 = nn.Linear(hidden_size,1)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        # out = out.view(-1,64) # wjy
        out = out.view(out.size(0),-1)
        out = F.relu(self.fc1(out))
        out = torch.sigmoid(self.fc2(out))
        # print(out.shape)
        return out

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('BatchNorm') != -1:
        m.weight.data.fill_(1)
        m.bias.data.zero_()
    elif classname.find('Linear') != -1:
        n = m.weight.size(1)
        m.weight.data.normal_(0, 0.01)
        m.bias.data = torch.ones(m.bias.data.size())

In [4]:
# Hyper Parameters
FEATURE_DIM = 64
RELATION_DIM = 8
CLASS_NUM = 5 # way
SAMPLE_NUM_PER_CLASS = 1 # shot
BATCH_NUM_PER_CLASS = 15 # support?
EPISODE = 500000
TEST_EPISODE = 600
LEARNING_RATE = 0.001
GPU = 1
HIDDEN_UNIT = 10

# TRAIN_NUM = 6
TRAIN_NUM = 12
VALID_NUM = 6
TEST_NUM = 6
snr = 18
sampling_size = 100

In [5]:
mods = ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '16APSK', '32APSK', '64APSK', '128APSK',
'16QAM', '32QAM', '64QAM', '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK', 'OQPSK']
snrs = [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
# classes = ['32PSK','16APSK','32QAM','FM','GMSK','32APSK','OQPSK','8ASK','BPSK','8PSK','AM-SSB-SC','4ASK',
# '16PSK','64APSK','128QAM','128APSK','AM-DSB-SC','AM-SSB-WC','64QAM','QPSK','256QAM','AM-DSB-WC','OOK','16QAM']

class MyLabelEncoder(LabelEncoder):
    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y).unique()
        return self

le = MyLabelEncoder()
le.fit(mods)

n_mod = len(mods) # 24
n_snr = len(snrs) # 26
n_samples = 4096
len_samples = 1024
dir_path = './data/'
classes = range(n_mod)

with h5py.File(dir_path+str(snr)+".hdf5", "r") as f:
  # I = f.get('I').value
  # Q = f.get('Q').value
  # IQ = np.stack((I,Q),axis=1) # (98304, 2, 1024)
  IQ = np.stack((f.get('I').value,f.get('Q').value),axis=1)
  label=(f.get('label').value).reshape(-1)

rp = RecurrencePlot(dimension=2, time_delay=940, threshold=None, percentage=10, flatten=False)
# image_size = n_timestamps - (dimension - 1) * time_delay

# _, sample_IQ, _, sample_label = train_test_split(IQ, label, test_size=n_mod*sampling_size, stratify=label)
_, sample_IQ, _, Y = train_test_split(IQ, label, test_size=n_mod*sampling_size, stratify=label)
temp = np.split(sample_IQ, 2, axis=1)
sample_I = np.array(temp)[0].reshape(-1,len_samples)
sample_Q = np.array(temp)[1].reshape(-1,len_samples)
rp_I = rp.transform(sample_I)
rp_Q = rp.transform(sample_Q)
# rp_IQ = np.stack((rp_I,rp_Q),axis=1)
# X = rp_IQ
X = np.stack((rp_I,rp_Q),axis=1)
len_samples = X.shape[2]
# Y = le.inverse_transform(np.array(Y, dtype=int))

# import collections
# print(collections.Counter(sample_label))

<ipython-input-5-a50b042f5d1d>:26: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  IQ = np.stack((f.get('I').value,f.get('Q').value),axis=1)
<ipython-input-5-a50b042f5d1d>:27: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  label=(f.get('label').value).reshape(-1)


In [6]:
# Step 1: init data folders
# init neural networks
metatrain_folders, metavalid_folders, metatest_folders = mini_imagenet_folders() # load folder root
# metatrain_folders, metatest_folders = mini_imagenet_folders() # load folder root

# Step 2: init neural networks
feature_encoder = CNNEncoder()
relation_network = RelationNetwork(FEATURE_DIM,RELATION_DIM)

feature_encoder.apply(weights_init)
relation_network.apply(weights_init)

feature_encoder.cuda(GPU)
relation_network.cuda(GPU)

feature_encoder_optim = torch.optim.Adam(feature_encoder.parameters(),lr=LEARNING_RATE)
feature_encoder_scheduler = StepLR(feature_encoder_optim,step_size=100000,gamma=0.5)
relation_network_optim = torch.optim.Adam(relation_network.parameters(),lr=LEARNING_RATE)
relation_network_scheduler = StepLR(relation_network_optim,step_size=100000,gamma=0.5)

if os.path.exists(str("./models/miniimagenet_feature_encoder_" + str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")):
    feature_encoder.load_state_dict(torch.load(str("./models/miniimagenet_feature_encoder_" + str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")))
    print("load feature encoder success")
if os.path.exists(str("./models/miniimagenet_relation_network_"+ str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")):
    relation_network.load_state_dict(torch.load(str("./models/miniimagenet_relation_network_"+ str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")))
    print("load relation network success")
    
# Step 3: build graph
warnings.filterwarnings(action='ignore')

enc = OneHotEncoder()
# print("Training...")

last_accuracy = 0.0
for episode in range(EPISODE):
    feature_encoder_scheduler.step(episode)
    relation_network_scheduler.step(episode)

    task = MiniImagenetTask(metatrain_folders,CLASS_NUM,SAMPLE_NUM_PER_CLASS,BATCH_NUM_PER_CLASS)
    sample_dataloader = get_mini_imagenet_data_loader(task,num_per_class=SAMPLE_NUM_PER_CLASS,split="train",shuffle=False)
    batch_dataloader = get_mini_imagenet_data_loader(task,num_per_class=BATCH_NUM_PER_CLASS,split="test",shuffle=True)

    samples,sample_labels = sample_dataloader.__iter__().next() #25*3*84*84
    # print('sample shape is {}'.format(samples.shape))
    batches,batch_labels = batch_dataloader.__iter__().next()
    # print('batch shape is {}'.format(batches.shape))


    # calculate features
    sample_features = feature_encoder(Variable(samples).cuda(GPU)) # 25*64*19*19
    batch_features = feature_encoder(Variable(batches).cuda(GPU)) # 20x64*5*5

    # calculate relations
    # each batch sample link to every samples to calculate relations
    # to form a 100x128 matrix for relation network
    sample_features_ext = sample_features.unsqueeze(0).repeat(BATCH_NUM_PER_CLASS*CLASS_NUM,1,1,1,1)
    batch_features_ext = batch_features.unsqueeze(0).repeat(SAMPLE_NUM_PER_CLASS*CLASS_NUM,1,1,1,1)
    batch_features_ext = torch.transpose(batch_features_ext,0,1)
    relation_pairs = torch.cat((sample_features_ext,batch_features_ext),2).view(-1,FEATURE_DIM*2,19,19)
    relations = relation_network(relation_pairs).view(-1,CLASS_NUM*SAMPLE_NUM_PER_CLASS)

    mse = nn.MSELoss().cuda(GPU)
    # one_hot_labels = Variable(torch.zeros(BATCH_NUM_PER_CLASS*CLASS_NUM, CLASS_NUM).scatter_(1, batch_labels.view(-1,1), 1)).cuda(GPU)
    one_hot_labels = Variable(torch.tensor(enc.fit_transform(batch_labels.view(-1, 1)).toarray())).float().cuda(GPU)
    # print(one_hot_labels.shape)
    loss = mse(relations,one_hot_labels)

    # training
    feature_encoder.zero_grad()
    relation_network.zero_grad()

    loss.backward()

    torch.nn.utils.clip_grad_norm(feature_encoder.parameters(),0.5)
    torch.nn.utils.clip_grad_norm(relation_network.parameters(),0.5)

    feature_encoder_optim.step()
    relation_network_optim.step()


    if (episode+1)%5000 == 0:
            print("episode:",episode+1,"loss",loss.item())

    if episode%5000 == 0:
        # test
        # print("Testing...")
        accuracies = []
        for i in range(TEST_EPISODE):
            total_rewards = 0
            counter = 0
            # task = MiniImagenetTask(metatest_folders,CLASS_NUM,SAMPLE_NUM_PER_CLASS,SAMPLE_NUM_PER_CLASS)
            task = MiniImagenetTask(metavalid_folders,CLASS_NUM,SAMPLE_NUM_PER_CLASS,SAMPLE_NUM_PER_CLASS)
            
            sample_dataloader = get_mini_imagenet_data_loader(task,num_per_class=1,split="train",shuffle=False)
            test_dataloader = get_mini_imagenet_data_loader(task,SAMPLE_NUM_PER_CLASS,split="test",shuffle=True)
            
            sample_images,sample_labels = sample_dataloader.__iter__().next()
            for test_images,test_labels in test_dataloader:
                batch_size = test_labels.shape[0]
                # calculate features
                sample_features = feature_encoder(Variable(sample_images).cuda(GPU)) # 5x64
                test_features = feature_encoder(Variable(test_images).cuda(GPU)) # 20x64

                # calculate relations
                # each batch sample link to every samples to calculate relations
                # to form a 100x128 matrix for relation network
                sample_features_ext = sample_features.unsqueeze(0).repeat(batch_size,1,1,1,1)
                test_features_ext = test_features.unsqueeze(0).repeat(1*CLASS_NUM,1,1,1,1)
                test_features_ext = torch.transpose(test_features_ext,0,1)
                relation_pairs = torch.cat((sample_features_ext,test_features_ext),2).view(-1,FEATURE_DIM*2,19,19)
                relations = relation_network(relation_pairs).view(-1,CLASS_NUM)

                _,predict_labels = torch.max(relations.data,1)
                # print(predict_labels)
                # print(test_labels)

                rewards = [1 if predict_labels[j]==test_labels[j] else 0 for j in range(batch_size)]

                total_rewards += np.sum(rewards)
                counter += batch_size

            accuracy = total_rewards/1.0/counter
            accuracies.append(accuracy)

        test_accuracy,h = mean_confidence_interval(accuracies)

        print("test accuracy:",test_accuracy,"h:",h)

        if test_accuracy > last_accuracy:

            # save networks
            torch.save(feature_encoder.state_dict(),str("./models/miniimagenet_feature_encoder_" + str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl"))
            torch.save(relation_network.state_dict(),str("./models/miniimagenet_relation_network_"+ str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl"))

            print("save networks for episode:",episode)

            last_accuracy = test_accuracy

test accuracy: 0.19233333333333336 h: 0.022122918809038
save networks for episode: 0
test accuracy: 0.18799999999999997 h: 0.02718716088249977
test accuracy: 0.19699999999999998 h: 0.026296444584585826
save networks for episode: 10000
test accuracy: 0.23233333333333334 h: 0.028312397130955534
save networks for episode: 15000
test accuracy: 0.257 h: 0.030126395350639986
save networks for episode: 20000
test accuracy: 0.36333333333333334 h: 0.03178742256345722
save networks for episode: 25000
test accuracy: 0.5156666666666667 h: 0.030900530222644697
save networks for episode: 30000
test accuracy: 0.587 h: 0.029595933015819673
save networks for episode: 35000
test accuracy: 0.7896666666666666 h: 0.024263266562356105
save networks for episode: 40000
test accuracy: 0.8586666666666666 h: 0.022144786073730976
save networks for episode: 45000
test accuracy: 0.9136666666666667 h: 0.01727514979874564
save networks for episode: 50000
test accuracy: 0.8986666666666667 h: 0.01772617736994118
test a

In [ ]:
# # Hyper Parameters
# FEATURE_DIM = 64
# RELATION_DIM = 8
# CLASS_NUM = 5 # way
# SAMPLE_NUM_PER_CLASS = 1 # shot
# BATCH_NUM_PER_CLASS = 15 # support?
EPISODE = 10
TEST_EPISODE = 1000
# LEARNING_RATE = 0.001
# GPU = 1
# HIDDEN_UNIT = 10

# TRAIN_NUM = 6
# TRAIN_NUM = 14
# VALID_NUM = 5
# TEST_NUM = 5
# snr = 30
# sampling_size = 1000

# # metatrain_folders,metatest_folders = mini_imagenet_folders()
# metatrain_folders, metavalid_folders, metatest_folders = mini_imagenet_folders() # load folder root

feature_encoder = CNNEncoder()
relation_network = RelationNetwork(FEATURE_DIM,RELATION_DIM)

feature_encoder.cuda(GPU)
relation_network.cuda(GPU)

feature_encoder_optim = torch.optim.Adam(feature_encoder.parameters(),lr=LEARNING_RATE)
feature_encoder_scheduler = StepLR(feature_encoder_optim,step_size=100000,gamma=0.5)
relation_network_optim = torch.optim.Adam(relation_network.parameters(),lr=LEARNING_RATE)
relation_network_scheduler = StepLR(relation_network_optim,step_size=100000,gamma=0.5)

if os.path.exists(str("./models/miniimagenet_feature_encoder_" + str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")):
    feature_encoder.load_state_dict(torch.load(str("./models/miniimagenet_feature_encoder_" + str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")))
    print("load feature encoder success")
if os.path.exists(str("./models/miniimagenet_relation_network_"+ str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")):
    relation_network.load_state_dict(torch.load(str("./models/miniimagenet_relation_network_"+ str(CLASS_NUM) +"way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl")))
    print("load relation network success")

total_accuracy = 0.0
for episode in range(EPISODE):
    accuracies = []
    for i in range(TEST_EPISODE):
        total_rewards = 0
        counter = 0
        task = MiniImagenetTask(metatest_folders,CLASS_NUM,SAMPLE_NUM_PER_CLASS,SAMPLE_NUM_PER_CLASS)
        sample_dataloader = get_mini_imagenet_data_loader(task,num_per_class=1,split="train",shuffle=False)
        test_dataloader = get_mini_imagenet_data_loader(task,SAMPLE_NUM_PER_CLASS,split="test",shuffle=True)
        
        sample_images,sample_labels = sample_dataloader.__iter__().next()
        for test_images,test_labels in test_dataloader:
            batch_size = test_labels.shape[0]

            sample_features = feature_encoder(Variable(sample_images).cuda(GPU)) # 5x64
            test_features = feature_encoder(Variable(test_images).cuda(GPU)) # 20x64

            sample_features_ext = sample_features.unsqueeze(0).repeat(batch_size,1,1,1,1)
            test_features_ext = test_features.unsqueeze(0).repeat(1*CLASS_NUM,1,1,1,1)
            test_features_ext = torch.transpose(test_features_ext,0,1)
            relation_pairs = torch.cat((sample_features_ext,test_features_ext),2).view(-1,FEATURE_DIM*2,19,19)
            relations = relation_network(relation_pairs).view(-1,CLASS_NUM)

            _,predict_labels = torch.max(relations.data,1)

            rewards = [1 if predict_labels[j]==test_labels[j] else 0 for j in range(batch_size)]

            total_rewards += np.sum(rewards)
            counter += batch_size

        accuracy = total_rewards/1.0/counter
        accuracies.append(accuracy)

    test_accuracy,h = mean_confidence_interval(accuracies)
    print("test accuracy:",test_accuracy,"h:",h)
    total_accuracy += test_accuracy

print("aver_accuracy:",total_accuracy/EPISODE)

test accuracy: 0.8164 h: 0.015742259289923437
test accuracy: 0.8192 h: 0.01577898452468244
test accuracy: 0.8348 h: 0.015417897283453195
test accuracy: 0.8400000000000001 h: 0.014660649748674218
test accuracy: 0.8232 h: 0.015169993786858568
test accuracy: 0.8238000000000001 h: 0.015130786888387145
test accuracy: 0.8268 h: 0.015228323784046106


KeyboardInterrupt: 